# Ollama to set up LLM on premises

To work with `ollama` you can [download](https://ollama.com/download) and install its app or you can also mount a [docker image containing it](https://hub.docker.com/r/ollama/ollama).  In either case you will get an [API](https://github.com/ollama/ollama/blob/main/docs/api.md) with which you can interact with using its own [ollama python library](https://pypi.org/project/ollama/) or another third party tool such as [langchain](https://python.langchain.com/v0.2/docs/how_to/local_llms/#ollama).

 * When the app is running you must fetch a model from this list of options: e.g., ollama pull gemma2:2b
    and all models are automatically served on *localhost:11434*
   
 * The LLM model that we gonna load must already be fetched by **pull**
 * When the model is unused the memory used is released (less than 5 minutes)


In [32]:
from IPython.display import display, Markdown  # to see better the output text


## From ollama library

In [47]:
# %pip install ollama
from ollama import Client

client = Client(host='http://localhost:11434')

### To pull a model

In [ ]:
client.pull(model='gemma2:2b')

In [46]:
client.show(model='gemma2:2b')['details']

{'parent_model': '',
 'format': 'gguf',
 'family': 'gemma2',
 'families': ['gemma2'],
 'parameter_size': '2.6B',
 'quantization_level': 'Q4_0'}

### To delete a model
https://github.com/ollama/ollama/blob/main/docs/api.md#delete-a-model

In [ ]:
client.delete(model='gemma2:2b')

### To make predictions

In [33]:
response = client.chat(model="gemma2:2b", options={"temperature": 0.0}, messages=[
    {
        'role': 'user',
        'content': 'Who was the first man on the moon?',
    },
])
Markdown(response['message']['content'])

**Neil Armstrong** was the first man to walk on the Moon. 

He achieved this historic feat during the Apollo 11 mission on July 20, 1969.  His famous words upon stepping onto the lunar surface were: "That's one small step for [a] man, one giant leap for mankind."


## Using Gemma2 in Ollama from langchain
* https://ollama.com/library/gemma2:2b

* It takes 3GB of GPU to make the inference

In [67]:
# %pip install -qU langchain_ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma2:2b", temperature=0.0)

Markdown(llm.invoke("'Who was the first man on the moom?")) # moom: misspelling

The first person to walk on the Moon was **Neil Armstrong**, an American astronaut, on July 20, 1969, during the Apollo 11 mission.  


In [35]:
Markdown(llm.invoke("hi, my name is alejandro"))

Hi Alejandro! 👋  It's nice to meet you. 😊 

What can I help you with today? 😄 


In [78]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Keep the answer concise.\n Question: {query}""")

chain = prompt | llm

In [36]:
out = chain.invoke({"query": "Tell me a funny joke about mathematics"})
Markdown(out)

Why is six afraid of seven? 

Because seven eight nine! 😂 


In [37]:
out = chain.invoke({"query": "tell me about paris?"})
Markdown(out)

Paris is the capital of France, known for its iconic landmarks like the Eiffel Tower and Louvre Museum. It boasts beautiful architecture, rich history, delicious cuisine, and vibrant culture. 


In [79]:
text = (
"Formally, a database refers to a set of related data accessed through the use of a" 
"database management system (DBMS), which is an integrated set of computer software that allows"
"users to interact with one or more databases and provides access to all of the data contained in "
"the database (although restrictions may exist that limit access to particular data). The DBMS "
"provides various functions that allow entry, storage and retrieval of large quantities of "
"information and provides ways to manage how that information is organized. High-performance "
"computing is critical for the processing and analysis of data. One particularly widespread "
"approach to computing for data engineering is dataflow programming, in which the computation is "
"represented as a directed graph (dataflow graph); nodes are the operations, and edges represent "
"the flow of data. Popular implementations include Apache Spark, and the deep learning specific "
"TensorFlow. More recent implementations, such as Differential/Timely Dataflow, have used "
"incremental computing for much more efficient data processing."
)
out = chain.invoke(
    {"query": f"What is the last sentence of the following text? \n ```{text} ```"})
Markdown(out)

"More recent implementations, such as Differential/Timely Dataflow, have used incremental computing for much more efficient data processing." 


### Translation and summary

In [69]:
Markdown(llm.invoke(f"can you translate to spanish the following text: {text}"))

Formalmente, una base de datos se refiere a un conjunto de datos relacionados que se accede mediante un sistema de gestión de bases de datos (DBMS), que es un conjunto integrado de software informático que permite a los usuarios interactuar con una o más bases de datos y proporciona acceso a toda la información contenida en la base de datos (aunque pueden existir restricciones que limitan el acceso a ciertos datos). El DBMS proporciona varias funciones que permiten la entrada, almacenamiento y recuperación de grandes cantidades de información y proporciona formas de gestionar cómo está organizada esa información. La computación de alto rendimiento es crucial para el procesamiento y análisis de datos. Un enfoque particularmente extendido para la computación de ingeniería de datos es el programación de flujo de datos, en el cual la computación se representa como un gráfico dirigido (gráfico de flujo de datos); los nodos son las operaciones y los bordes representan el flujo de datos. Implementaciones populares incluyen Apache Spark y TensorFlow, específico para aprendizaje profundo. Implementaciones más recientes, como Differential/Timely Dataflow, han utilizado la computación incremental para un procesamiento de datos mucho más eficiente. 


In [70]:
Markdown(llm.invoke(f"can you summary the following text in two sentences: {text}"))

A database is a structured collection of related information accessed through a DBMS, which manages and organizes data while allowing users to interact with it.  Data engineering utilizes techniques like dataflow programming and advanced algorithms like TensorFlow to process and analyze large datasets efficiently. 


In [29]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Keep the answer concise."
    "\n\n"
    "{context}"
)

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)
rag_chain = rag_prompt|llm

### Answer with context

In [39]:
Markdown(rag_chain.invoke(
    {"query": "What is  dataflow programming? ", "context": text}))

Dataflow programming represents computation as a directed graph where nodes are operations and edges represent data flow. 


### Answer without context

In [40]:
Markdown(llm.invoke("What is  dataflow programming?"))

## Dataflow Programming: A Stream of Data

Dataflow programming is a paradigm where **data flows through the program**, driven by events and transformations. It's like a river, with data as water flowing from one point to another, undergoing various processes along the way. 

**Here's how it works:**

1. **Data Sources:**  The journey starts with data sources (like sensors, databases, or files).
2. **Processing Units:** These are "nodes" that perform operations on the data. Think of them as factories where raw materials are transformed into finished products. 
3. **Connections:** Data flows from source to processing unit and then to other units, forming a network of interconnected nodes. 
4. **Data Transformations:**  Processing units apply functions or algorithms to the data, changing its form and content. This is like adding value to raw materials in our factory analogy.
5. **Outputs:** The transformed data flows out from processing units, ready for further use or storage.

**Key Features of Dataflow Programming:**

* **Parallelism:**  Dataflow programs can process multiple data streams simultaneously, making them highly efficient and scalable. 
* **Event-Driven:**  The program reacts to events (like new data arriving) and executes the necessary transformations accordingly. This makes it responsive and adaptable.
* **Declarative:** You describe what you want the program to do without specifying how to achieve it. The system handles the details of execution.

**Benefits of Dataflow Programming:**

* **Increased Efficiency:**  Parallel processing leads to faster execution times, especially for complex tasks involving large datasets. 
* **Simplified Development:**  Focus on data flow and transformations instead of low-level control, making development easier and more intuitive.
* **Flexibility:**  Adaptable to changing requirements as new data sources or processing units can be easily added.

**Examples of Dataflow Programming:**

* **Data Streaming:** Processing real-time data streams from sensors, IoT devices, or social media feeds. 
* **Machine Learning:** Training and deploying machine learning models on large datasets.
* **Signal Processing:** Analyzing audio, video, or sensor signals for various applications like image recognition or speech synthesis.


**In Summary:**

Dataflow programming is a powerful approach to building software that handles massive amounts of data efficiently and flexibly. It's ideal for tasks where speed, scalability, and adaptability are crucial. 


## Using Phi2:mini-4k in Ollama from langchain
* https://ollama.com/library/phi3:mini-4k
* It takes 6GB of GPU to make the inference

In [50]:
client.pull(model='phi3:mini-4k')

{'status': 'success'}

In [51]:
client.show(model='phi3:mini-4k')['details']

{'parent_model': '',
 'format': 'gguf',
 'family': 'phi3',
 'families': ['phi3'],
 'parameter_size': '3.8B',
 'quantization_level': 'Q4_K_M'}

In [76]:
# %pip install -qU langchain_ollama
from langchain_ollama import OllamaLLM

llm_phi = OllamaLLM(model="phi3:mini-4k", temperature=0.0)

Markdown(llm_phi.invoke("'Who was the first man on the moom?")) # moom: misspelling

The phrase "the first man on the moon" refers to astronaut Neil Armstrong, who became the first human to set foot on the lunar surface during NASA's Apollo 11 mission. This historic event took place on July 20, 1969.

In [54]:
Markdown(llm_phi.invoke("hi, my name is alejandro"))

Hello Alejandro! How can I assist you today?

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Keep the answer concise.\n Question: {query}""")

chain = prompt | llm_phi

In [56]:
out = chain.invoke({"query": "Tell me a funny joke about mathematics"})
Markdown(out)

Why did the math book look so sad? Because it had too many problems!

In [57]:
out = chain.invoke({"query": "tell me about paris?"})
Markdown(out)

Paris, known as "The City of Light," is a global center for art, fashion, gastronomy, and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. The Eiffel Tower, an iconic symbol of France, stands tall in its heart. Paris has been one of Europe's major cities since the Middle Avestrian Age (around AD 50). It was a center for education with institutions like Sorbonne University founded here during medieval times and later became known as "The City of Light" due to it being among the first large European cities to use gas street lighting.

In [77]:
text = (
"Formally, a database refers to a set of related data accessed through the use of a" 
"database management system (DBMS), which is an integrated set of computer software that allows"
"users to interact with one or more databases and provides access to all of the data contained in "
"the database (although restrictions may exist that limit access to particular data). The DBMS "
"provides various functions that allow entry, storage and retrieval of large quantities of "
"information and provides ways to manage how that information is organized. High-performance "
"computing is critical for the processing and analysis of data. One particularly widespread "
"approach to computing for data engineering is dataflow programming, in which the computation is "
"represented as a directed graph (dataflow graph); nodes are the operations, and edges represent "
"the flow of data. Popular implementations include Apache Spark, and the deep learning specific "
"TensorFlow. More recent implementations, such as Differential/Timely Dataflow, have used "
"incremental computing for much more efficient data processing."
)
out = chain.invoke(
    {"query": f"What is the last sentence of the following text? \n ```{text} ```"})
Markdown(out)

The text does not provide a last sentence explicitly; it ends with information about popular implementations of dataflow programming like Apache Spark and TensorFlow before mentioning newer approaches without concluding the paragraph or statement.

### Translation and summary

In [65]:
Markdown(llm_phi.invoke(f"can you translate to spanish the following text: {text}"))

Formalmente, un sistema de base de datos se refiere a un conjunto de datos relacionados accesibles mediante el uso de una Base de Datos Management System (DBMS), que es un conjunto integrado de software informático que permite la interacción con uno o más sistemas de bases de datos y proporciona acceso a toda la información contenida en ellos (si bien pueden existir restricciones que limitan el acceso a ciertos datos). El DBMS ofrece diversas funciones que permiten la entrada, almacenamiento y recuperación de grandes cantidades de información e incluye formas para gestionar cómo esa información está organizada. La computación en alta capacidad es crítica para el procesamiento y análisis de datos. Una aproximación particularmente extendida a la programación para ingeniería de datos es la programación basada en flujo, donde se representa la computación como un grafo dirigido (grafo de flujos); los nodos son las operaciones y las aristas representan el flujo de datos. Implementaciones populares incluyen Apache Spark e TensorFlow específico para aprendizaje profundo. Más recientemente, implementaciones como Differential/Timely Dataflow han utilizado la computación incremental para un procesamiento mucho más eficiente del mismo.

In [71]:
Markdown(llm_phi.invoke(f"can you summary the following text in two sentences: {text}"))

A database management system (DBMS) is a software that allows users to interact with databases by providing functions for entry, storage and retrieval of large quantities of information while managing its organization; high-performance computing plays an essential role in this process. Dataflow programming represents computation as directed graphs where nodes are operations and edges represent data flow—Apache Spark and TensorFlow being popular implementations, with newer ones like Differential/Timely Dataflow using incremental computing for efficient processing of large datasets.

In [60]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Keep the answer concise."
    "\n\n"
    "{context}"
)

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)
rag_chain = rag_prompt|llm_phi

### Answer with context

In [61]:
Markdown(rag_chain.invoke(
    {"query": "What is  dataflow programming? ", "context": text}))

Dataflow programming is a paradigm where computation models are represented by directed graphs (dataflow graphs), with nodes representing operations and edges indicating the flow of data between these operations. This approach allows parallel execution, as different parts of the graph can be processed simultaneously if they do not depend on each other's results. It enables efficient processing for large-scale computations in fields like big data analytics and machine learning by optimizing resource utilization and reducing latency through incremental computing techniques such as those used in Differential/Timely Dataflow implementations, which are more recent advanc employee satisfaction can be measured using various methods. One common approach is conducting surveys or questionnaires that assess employees' attitudes towards their work environment, job roles, and the organization itself. These tools often include questions about engagement levels, motivation, perceived support from management, opportunities for growth, recognition of achievements, work-life balance, and overall satisfaction with their jobs.

Another method is through interviews or focus groups that allow employees to discuss in more depth the factors contributing to their job satisfaction. This qualitative approach can provide deeper insights into employee sentiments and experiences within an organization. Additionally, performance metrics such as turnover rates, absenteeism, productivity levels, and quality of work output are indirect indicators of overall employee morale and engagement.

Organizations may also use more sophisticated tools like the Job Descriptive Index (JDI) or the Minnesota Satisfaction Questionnaire (MSQ), which have been developed specifically to measure job satisfaction across various dimensions systematically. These instruments are designed based on psychological theories of motivation and well-being, ensuring that they capture a comprehensive picture of employee attitudes towards their work life.

It's important for organizations to regularly assess employee morale as it can significantly impact productivity, retention rates, customer satisfaction, and overall organizational success.

### Answer without context

In [62]:
Markdown(llm_phi.invoke("What is  dataflow programming?"))

Dataflow programming is a model of computation where the program's control flow depends on the availability and state of various pieces of data. In this paradigm, programs are composed of nodes representing operations or computations that process input data to produce output results. The execution proceeds by passing values from one node (or operation) to another along a directed graph called a "dataflow network" until all the required inputs have been processed and their corresponding outputs can be produced.

The key characteristics of dataflow programming include:

1. Declarative nature: Dataflow programs are typically written in a declarative manner, where you specify what needs to be done rather than how it should be done (as opposed to imperative languages). This makes the code more readable and easier to understand since each operation is clearly defined with its inputs and outputs.
2. Asynchronous execution: Since dataflow programs are event-driven by nature, they can execute asynchronously without blocking other operations waiting for their turn in a sequential manner (as seen in traditional imperative programming). This allows better utilization of resources like CPUs or GPUs since multiple computations may be performed simultaneously.
3. Dynamic execution: Dataflow programs are dynamic and adaptable to changes, meaning that the program can adjust its behavior based on available data at runtime without requiring explicit reconfiguration by a programmer (as seen in static programming models). This makes it easier for developers to create flexible applications capable of handling various scenarios or inputs efficiently.
4. Parallelism: Dataflow programs are inherently parallelizable since different operations may be executed simultaneously, depending on the availability and dependencies between data nodes within their network structure. By exploiting this property effectively (e.g., using multi-core processors), developers can achieve significant performance improvements in computationally intensive tasks or large datasets processing applications like image/video analysis, machine learning algorithms, etc.
5. Highly modular: Dataflow programs are composed of independent nodes that represent discrete operations on data elements; this makes it easier to reuse and compose existing components (e.g., libraries) within new systems without worrying about the underlying implementation details or dependencies between different parts of a program's execution flow.
6. Efficient resource utilization: Since each node in a dataflow network operates independently, only those nodes that have available input data will be executed at any given time (as opposed to traditional programming models where all operations may need to execute sequentially). This leads to better CPU/GPU usage and reduced power consumption when processing large datasets or complex computations.
7. Easy debugging: Debugging a dataflow program can often involve tracing the flow of input values through various nodes in its network structure until their corresponding outputs are produced (as opposed to traditional programming models where bugs may be harder to pinpoint due to interdependencies between different parts). This makes it easier for developers and users alike to understand how an application works or identify potential issues within a given system.
8. Scalability: Dataflow programs can scale well across multiple processing units (e. employee) since each node operates independently, allowing them to be distributed over various hardware resources without affecting the overall execution flow of other nodes in their network structure. This makes it easier for developers and organizations alike to build scalable systems capable of handling large datasets or computationally intensive tasks efficiently while maintaining high performance levels even under heavy loads (e.g., real-time data processing applications).
9. Flexibility: Dataflow programming models are highly flexible since they allow programmers to define their own custom operations and combine them in various ways within a network structure depending on the specific requirements of an application or problem domain being addressed by developers/organizations alike (e.g., creating specialized algorithms for image processing tasks).
10. Ease-of-use: Dataflow programming models are often easier to learn than traditional imperative languages since they focus more on specifying what needs to be done rather than how it should be done; this makes them suitable for beginners or non-programmers who want to create simple applications without having extensive knowledge of computer science concepts like algorithms, data structures etc.
In summary, Dataflow programming is a powerful paradigm that offers numerous benefits such as parallelism, modularity, efficient resource utilization and scalability among others which make it an attractive choice for developing complex systems capable of handling large datasets or computationally intensive tasks efficiently while maintaining high performance levels even under heavy loads.